In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from google.colab import drive
import concurrent.futures
import csv
from sklearn.model_selection import StratifiedKFold

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta de la carpeta con las imágenes
path = '/content/drive/MyDrive/Estudios/Máster IIR/Segundo cuatrimestre/Proyecto VAI-AAI/chest_xray'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

radio = 3
n_puntos = 8 * radio

def procesar_imagen(imagen_path, size, etiqueta):
    imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f'No se pudo leer la imagen: {imagen_path}')
        return None, None

    imagen = cv2.resize(imagen, size)
    lbp = local_binary_pattern(imagen, n_puntos, radio, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, n_puntos + 3),
                             range=(0, n_puntos + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)

    return hist, etiqueta

def cargar_datos(ruta, size=(128, 128)):
    datos = []
    etiquetas = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for i, cat in enumerate(categorias):
            carpeta = join(ruta, cat)
            archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
            for archivo in archivos:
                imagen_path = join(carpeta, archivo)
                futures.append(executor.submit(procesar_imagen, imagen_path, size, i))

        for future in concurrent.futures.as_completed(futures):
            hist, label = future.result()
            if hist is not None:
                datos.append(hist)
                etiquetas.append(label)

    return np.array(datos), np.array(etiquetas)

# Cargar datos
datos, etiquetas = cargar_datos(join(path, 'train'))

# Preparar archivo CSV para guardar los resultados
nombre_archivo_csv = '/content/drive/MyDrive/Estudios/Máster IIR/Segundo cuatrimestre/Proyecto VAI-AAI/first_version/cv_knn_lbp.csv'
with open(nombre_archivo_csv, 'a', newline='') as archivo_csv:
    es_vacio = archivo_csv.tell() == 0
    fieldnames = ['Fold', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Specificity', 'AUC']
    writer = csv.DictWriter(archivo_csv, fieldnames=fieldnames)
    if es_vacio:
      writer.writeheader()

    cv = StratifiedKFold(n_splits=5)
    fold = 0
    for train_index, test_index in cv.split(datos, etiquetas):
        fold += 1
        X_train, X_test = datos[train_index], datos[test_index]
        y_train, y_test = etiquetas[train_index], etiquetas[test_index]

        modelo = KNeighborsClassifier(n_neighbors=5)
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)

        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred, average='macro')
        recall = metrics.recall_score(y_test, y_pred, average='macro')
        f1 = metrics.f1_score(y_test, y_pred, average='macro')

       # Cálculo de Specificity
        cm = confusion_matrix(y_test, y_pred)
        specificity = cm[0,0] / (cm[0,0] + cm[0,1])

        # Cálculo de AUC
        if hasattr(modelo, "decision_function"):
            y_scores = modelo.decision_function(X_test)
        else:
            y_scores = modelo.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_scores)

        # Escribir los resultados para cada fold
        writer.writerow({
            'Fold': fold,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'Specificity': specificity,
            'AUC': auc
        })

print("La evaluación ha finalizado y los resultados se han guardado en el archivo CSV.")


# Cargar datos de prueba
datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'))

# Predecir etiquetas para el conjunto de prueba
etiquetas_predichas_prueba = modelo.predict(datos_prueba)

# Calcular y mostrar métricas de rendimiento para el conjunto de prueba
exactitud_prueba = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas_prueba)
cm_prueba = confusion_matrix(etiquetas_prueba, etiquetas_predichas_prueba)
sensibilidad_prueba = recall_score(etiquetas_prueba, etiquetas_predichas_prueba)
especificidad_prueba = cm_prueba[0,0] / (cm_prueba[0,0] + cm_prueba[0,1])
precision_prueba = precision_score(etiquetas_prueba, etiquetas_predichas_prueba)
f1_prueba = f1_score(etiquetas_prueba, etiquetas_predichas_prueba)

# Calcular AUC para el conjunto de prueba
if hasattr(modelo, "decision_function"):
    scores_prueba = modelo.decision_function(datos_prueba)
else:
    scores_prueba = modelo.predict_proba(datos_prueba)[:, 1]
auc_prueba = roc_auc_score(etiquetas_prueba, scores_prueba)

with open(nombre_archivo_csv, 'a', newline='') as archivo_csv:  # 'a' para añadir al archivo existente
    fieldnames = ['Fold', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Specificity', 'AUC']
    writer = csv.DictWriter(archivo_csv, fieldnames=fieldnames)

    # No necesitamos volver a escribir el encabezado, asumiendo que estamos añadiendo al archivo existente

    # Escribir resultados del conjunto de prueba en el archivo CSV
    writer.writerow({
        'Fold': 'Test',
        'Accuracy': exactitud_prueba * 100,
        'Precision': precision_prueba * 100,
        'Recall': sensibilidad_prueba * 100,
        'F1-Score': f1_prueba,
        'Specificity': especificidad_prueba * 100,
        'AUC': auc_prueba
    })

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
La evaluación ha finalizado y los resultados se han guardado en el archivo CSV.
